<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Dependencies" data-toc-modified-id="Dependencies-1">Dependencies</a></span></li><li><span><a href="#Paths" data-toc-modified-id="Paths-2">Paths</a></span></li><li><span><a href="#Main" data-toc-modified-id="Main-3">Main</a></span><ul class="toc-item"><li><span><a href="#CBTTC-MAF-summary" data-toc-modified-id="CBTTC-MAF-summary-3.1">CBTTC MAF summary</a></span></li><li><span><a href="#GDC-MAF-summary" data-toc-modified-id="GDC-MAF-summary-3.2">GDC MAF summary</a></span></li><li><span><a href="#DKFZ" data-toc-modified-id="DKFZ-3.3">DKFZ</a></span></li><li><span><a href="#Calculate-Tumour-Mutation-Burden-(TMB)" data-toc-modified-id="Calculate-Tumour-Mutation-Burden-(TMB)-3.4">Calculate Tumour Mutation Burden (TMB)</a></span></li></ul></li></ul></div>

Calculation of TMB from overlapped calls of strelka2 and mutect2  and VAF > 0.03 in CBTTC dataset and mutect2 calls with VAF > 0.03 in GDC dataset

# Dependencies

In [1]:
library(plyr)

# Paths

In [2]:
manifestpath <- "/Users/anabbi/OneDrive - UHN/Documents/IPD2/Manifests/"
datapath <- "/Users/anabbi/OneDrive - UHN/Documents/IPD2/Data/"
plotpath <- "/Users/anabbi/OneDrive - UHN/Documents/IPD2/Plots/"

# Main

In [3]:
load(paste0(datapath, "ESTIMATE/estimate_manifest_primary_clean_final.RData"))

In [4]:
table(estimate_manifest_primary_clean$recurrence)


  0   1 
555 217 

## CBTTC MAF summary 

post-recoding post-overlap post-VAF filter

In [6]:
CBTTC_mut <- read.table(file = paste0(datapath,"snv/CBTTC_mafsummary_overlapped_3VAF.txt"),sep = "\t", 
                        check.names = F,header = TRUE, stringsAsFactors = FALSE)

In [7]:
head(CBTTC_mut[order(CBTTC_mut$total, decreasing = T),])

,Tumor_Sample_Barcode,Frame_Shift_Del,Frame_Shift_Ins,In_Frame_Del,In_Frame_Ins,Missense_Mutation,Nonsense_Mutation,Nonstop_Mutation,total,sample_id
,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<chr>
1,BS_8AY2GM4G,21,21,0,1,9555,572,5,10175,7316-2085
2,BS_20TBZG09,17,31,0,1,9213,534,5,9801,7316-515
3,BS_02YBZSBY,9,165,0,0,8170,470,1,8815,7316-2189
4,BS_D0T6V861,9,154,0,0,8018,457,1,8639,7316-2189
5,BS_VW4XN9Y7,3,111,0,1,5563,266,7,5951,7316-2640
6,BS_F0GNWEJJ,42,14,9,0,717,25,0,807,7316-3311


In [8]:
dim(CBTTC_mut)

[1] 796  10

In [9]:
CBTTC <- estimate_manifest_primary_clean[estimate_manifest_primary_clean$group == "CBTTC",]

In [10]:
dim(CBTTC)

[1] 579  21

In [11]:
length(CBTTC_mut$sample_id[CBTTC_mut$sample_id %in% CBTTC$sample_id])

[1] 470

In [12]:
CBTTC_mut_subset <- CBTTC_mut[,c("sample_id", "Frame_Shift_Del", "Frame_Shift_Ins","In_Frame_Del", "In_Frame_Ins",
                                 "Missense_Mutation", "Nonsense_Mutation", "Nonstop_Mutation", "total")]

In [13]:
head(CBTTC_mut_subset)

,sample_id,Frame_Shift_Del,Frame_Shift_Ins,In_Frame_Del,In_Frame_Ins,Missense_Mutation,Nonsense_Mutation,Nonstop_Mutation,total
,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,7316-2085,21,21,0,1,9555,572,5,10175
2,7316-515,17,31,0,1,9213,534,5,9801
3,7316-2189,9,165,0,0,8170,470,1,8815
4,7316-2189,9,154,0,0,8018,457,1,8639
5,7316-2640,3,111,0,1,5563,266,7,5951
6,7316-3311,42,14,9,0,717,25,0,807


In [14]:
mafsum_CBTTC_estimate_manifest <- merge(CBTTC, CBTTC_mut_subset, by = "sample_id")

In [15]:
dim(mafsum_CBTTC_estimate_manifest)

[1] 470  29

In [16]:
head(mafsum_CBTTC_estimate_manifest)

,sample_id,aliquot_id,StromalScore,ImmuneScore,ESTIMATEScore,name,case_id,gender,ethnicity,race,⋯,sample_type,age_class,Frame_Shift_Del,Frame_Shift_Ins,In_Frame_Del,In_Frame_Ins,Missense_Mutation,Nonsense_Mutation,Nonstop_Mutation,total
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,7316-10,717108,948.2224,1027.2452,1975.4676,cae75655-bec6-4ab6-af6f-e286f4cae404.rsem.genes.results,C15129,Female,Not Hispanic or Latino,African,⋯,Initial CNS Tumor,Young adults,0,0,0,0,4,0,0,4
2,7316-101,588338,-844.5791,-289.5254,-1134.1044,f52e9f4d-6f98-41e6-872f-107137f59980.rsem.genes.results,C23862,Male,Not Hispanic or Latino,African,⋯,Initial CNS Tumor,Children,0,0,0,0,8,0,0,8
3,7316-1075,588351,-186.4694,669.7261,483.2567,f1ea5df0-95b3-436a-9168-036e482cb76f.rsem.genes.results,C291264,Male,Reported Unknown,Other,⋯,Initial CNS Tumor,NA,0,0,0,0,2,0,0,2
4,7316-1077,731885,-680.6484,-475.4882,-1156.1366,5a8eb07f-4d9b-437f-a44b-432e5dc6b1f8.rsem.genes.results,C291510,Male,Reported Unknown,Other,⋯,Initial CNS Tumor,Children,0,0,0,0,1,1,0,2
5,7316-1078,588352,-823.4809,-630.5107,-1453.9915,b38b70ba-1b14-49a8-9386-007bcb0ef2aa.rsem.genes.results,C291633,Male,Reported Unknown,Other,⋯,Initial CNS Tumor,Children,1,1,0,0,3,1,0,6
6,7316-1081,731897,-413.6641,879.1675,465.5034,1da1eb0c-f54d-4bce-8f02-55899fc8761f.rsem.genes.results,C410697,Male,Reported Unknown,Other,⋯,Initial CNS Tumor,Children,1,0,0,0,1,0,0,2


## GDC MAF summary 

post-recoding post-deduping post-VAF filter

In [17]:
GDC_mut <- read.table(file = paste0(datapath, "snv/GDC_mafsummary_3VAF_dedup.txt"),
                      sep = "\t", header = TRUE, stringsAsFactors = FALSE)

In [18]:
head(GDC_mut)

,Tumor_Sample_Barcode,Frame_Shift_Del,Frame_Shift_Ins,In_Frame_Del,In_Frame_Ins,Missense_Mutation,Nonsense_Mutation,Nonstop_Mutation,total,group
,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<chr>
1,TCGA-91-7771-01A,10,2,1,0,130,7,0,150,LUAD
2,TCGA-14-1823-01A,6,3,0,1,71,5,0,86,GBM
3,TCGA-GF-A769-01A,1,0,1,0,87,3,0,92,SKCM
4,TCGA-12-5295-01A,3,0,0,0,49,2,0,54,GBM
5,TCGA-KK-A7B2-01A,1,0,0,0,14,0,1,16,PRAD
6,TCGA-A6-A565-01A,63,13,0,0,246,16,0,338,COAD


In [19]:
dim(GDC_mut)

[1] 2503   10

In [20]:
GDC <- estimate_manifest_primary_clean[estimate_manifest_primary_clean$group == "TCGA" |
                                       estimate_manifest_primary_clean$group == "TARGET",]

In [21]:
dim(GDC)

[1] 2601   21

In [22]:
colnames(GDC_mut)[colnames(GDC_mut) == "Tumor_Sample_Barcode"] <- "sample_id"

In [23]:
GDC_mut_mutects_estimate_manifest <- merge(GDC, GDC_mut[,colnames(GDC_mut) != "group"],
                                                  by = "sample_id")

In [24]:
head(GDC_mut_mutects_estimate_manifest)

,sample_id,aliquot_id,StromalScore,ImmuneScore,ESTIMATEScore,name,case_id,gender,ethnicity,race,⋯,sample_type,age_class,Frame_Shift_Del,Frame_Shift_Ins,In_Frame_Del,In_Frame_Ins,Missense_Mutation,Nonsense_Mutation,Nonstop_Mutation,total
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,TARGET-30-PAIFXV-01A,TARGET-30-PAIFXV-01A-01R,-537.97880,483.8556,-54.1232,af82838c-9071-4dd7-8766-b818cf61cf82.bam,TARGET-30-PAIFXV,Female,Unknown,White,⋯,Primary Tumor,Children,1,0,0,0,26,1,0,28
2,TARGET-30-PAIPGU-01A,TARGET-30-PAIPGU-01A-01R,-319.22630,1240.6492,921.4229,b1d954e7-21a0-441e-a9c7-47e6a6454625.bam,TARGET-30-PAIPGU,Female,Unknown,White,⋯,Primary Tumor,NA,0,0,0,1,30,3,0,34
3,TARGET-30-PAITCI-01A,TARGET-30-PAITCI-01A-01R,-327.26820,200.6735,-126.5947,9704403e-aaf9-4a97-99bc-b2a9a9861767.bam,TARGET-30-PAITCI,Male,Unknown,White,⋯,Primary Tumor,Infants,0,5,0,3,7,4,0,19
4,TARGET-30-PAIVHE-01A,TARGET-30-PAIVHE-01A-01R,-15.87499,1033.6469,1017.7719,d8d30bd7-608d-4cbd-ba31-3f1fa6fefacf.bam,TARGET-30-PAIVHE,Male,not hispanic or latino,White,⋯,Primary Tumor,Children,0,0,0,0,23,0,0,23
5,TARGET-30-PAIXIF-01A,TARGET-30-PAIXIF-01A-01R,460.15050,978.1672,1438.3177,7c2b0108-1231-4543-a8bf-eb6644e76f41.bam,TARGET-30-PAIXIF,Male,Unknown,Unknown,⋯,Primary Tumor,Children,0,4,0,0,14,3,0,21
6,TARGET-30-PAKYZS-01A,TARGET-30-PAKYZS-01A-01R,-243.34180,-254.3727,-497.7145,ed5045df-3b18-41b8-ad62-a5a598ee3889.bam,TARGET-30-PAKYZS,Male,not hispanic or latino,White,⋯,Primary Tumor,Infants,0,1,0,0,4,0,0,5


In [25]:
table(GDC_mut_mutects_estimate_manifest$recurrence)


 0  1 
37 58 

In [26]:
dim(GDC)
dim(GDC_mut)
dim(GDC_mut_mutects_estimate_manifest)

[1] 2601   21

[1] 2503   10

[1] 2434   29

In [27]:
GDC_CBTTC_mut_manifest_estimate <- rbind(mafsum_CBTTC_estimate_manifest, GDC_mut_mutects_estimate_manifest)

## DKFZ

In [28]:
somatic_DKFZ <- read.table(file = paste0(datapath,"snv/somatic_SNVs_INDELs_inCommonRegions.for_TMB.txt"),
                        sep = "\t", header = FALSE, stringsAsFactors = FALSE)

In [29]:
colnames(somatic_DKFZ) <- c("sample_id", "total")
head(somatic_DKFZ)

,sample_id,total
,<chr>,<int>
1,H049-0CWK,3
2,H049-7SD3,4
3,H049-FG38,7
4,H049-GP41,3
5,H049-GWNW,7
6,H049-JVCT,48


In [30]:
DKFZ <- estimate_manifest_primary_clean[estimate_manifest_primary_clean$group == "DKFZ",]

In [31]:
DKFZ_mut_estimate_manifest <- merge(DKFZ,somatic_DKFZ,by = "sample_id")

In [32]:
dim(DKFZ_mut_estimate_manifest)

[1] 196  22

In [33]:
total_mut_manifest_estimate <- rbind.fill(GDC_CBTTC_mut_manifest_estimate,DKFZ_mut_estimate_manifest)

In [34]:
dim(GDC_CBTTC_mut_manifest_estimate)
dim(DKFZ_mut_estimate_manifest)
dim(total_mut_manifest_estimate)

[1] 2904   29

[1] 196  22

[1] 3100   29

In [35]:
table(total_mut_manifest_estimate$group)


 CBTTC   DKFZ TARGET   TCGA 
   470    196     98   2336 

In [37]:
table(total_mut_manifest_estimate$sample_type)


Initial CNS Tumor           Primary     Primary Tumor 
              470               196              2434 

## Calculate Tumour Mutation Burden (TMB)

size of bedfile used to subset variant calls (hglft_agil_VCR_Sure_S029_SeqCapV2V3_S04_pads_cexons_sorted_merge.be): 30028393

In [38]:
total_mut_manifest_estimate$mutpermb <- total_mut_manifest_estimate$total/30.028393

total_mut_manifest_estimate$snvpermb <- (total_mut_manifest_estimate$Missense_Mutation +
                                                 total_mut_manifest_estimate$Nonsense_Mutation + 
                                                 total_mut_manifest_estimate$Nonstop_Mutation)/30.028393

total_mut_manifest_estimate$indelpermb <- (total_mut_manifest_estimate$Frame_Shift_Del +
                                                 total_mut_manifest_estimate$Frame_Shift_Ins + 
                                                 total_mut_manifest_estimate$In_Frame_Del +
                                          total_mut_manifest_estimate$In_Frame_Ins)/30.028393

In [39]:
save(total_mut_manifest_estimate,
     file = paste0(datapath, "snv/total_mut_manifest_estimate.RData"))   